# 전처리

In [13]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('../data/merged_df_original.csv')

# '캐시워크'부터 '우체국보험'까지의 열 이름 목록 추출
start_col = '캐시워크'
end_col = '우체국보험'
cols_to_sum = df.loc[:, start_col:end_col].columns

df['m_usagestats'] = df[cols_to_sum].sum(axis=1)

df.drop(columns=cols_to_sum, inplace=True)

merged_df_cwj = df.copy()

merged_df_cwj.rename(columns={'m_wtb_rssi_x': 'm_wtb_rssi', 'm_wtb_rssi_y': 'm_wtw_rssi'}, inplace=True)

# CSV로 내보내기
merged_df_cwj.to_csv('../data/merged_df_cwj.csv', index=False)


print("✅ merged_df_cwj.csv 저장 완료")


✅ merged_df_cwj.csv 저장 완료


In [15]:
df = pd.read_csv('../data/merged_df_cwj.csv')

# 검사할 열 목록
cols_to_check = [
    'met_activity', 'm_wtb_rssi', 'm_wtw_rssi', 'heart_rate',
    'distance', 'latitude', 'longitude', 'altitude', 'speed', 'm_usagestats', 'w_light'
]

# 각 열의 결측치 개수 출력
missing_counts = df[cols_to_check].isnull().sum()
print("📊 결측치 개수:")
print(missing_counts)


📊 결측치 개수:
met_activity     2690
m_wtb_rssi      77337
m_wtw_rssi      22842
heart_rate      55846
distance        22852
latitude        16044
longitude       16044
altitude        16044
speed           16044
m_usagestats        0
w_light         19040
dtype: int64


### 결측치를 가장 가까운 이웃의 값(앞뒤 기준)으로 채움

In [16]:
#대상 열 목록
cols_to_check = [
    'met_activity', 'm_wtb_rssi', 'm_wtw_rssi', 'heart_rate',
    'distance', 'latitude', 'longitude', 'altitude', 'speed', 'm_usagestats', 'w_light'
]

# 결측치 보간 함수 (앞뒤 평균)
def fill_nearest_avg(series):
    forward = series.ffill()  # 앞쪽 값으로 채우기
    backward = series.bfill()  # 뒤쪽 값으로 채우기
    filled = series.copy()
    
    # 앞뒤 값이 모두 있는 경우 평균으로
    for i in series[series.isnull()].index:
        f, b = forward[i], backward[i]
        if pd.notnull(f) and pd.notnull(b):
            filled[i] = (f + b) / 2
        elif pd.notnull(f):
            filled[i] = f
        elif pd.notnull(b):
            filled[i] = b
    return filled

# 'burned_calories' 열의 결측치를 0으로 채우기
df['burned_calories'] = df['burned_calories'].fillna(0)

# 각 열에 대해 결측치 처리
for col in cols_to_check:
    df[col] = fill_nearest_avg(df[col])

# 확인용 출력 (선택사항)
print("✅ 결측치 보간 완료. 남은 결측치 수:")
print(df[cols_to_check].isnull().sum())

# 저장
df.to_csv('../data/merged_df_cwj_filled.csv', index=False)
print("📁 '../data/merged_df_cwj_filled.csv' 저장 완료")


✅ 결측치 보간 완료. 남은 결측치 수:
met_activity    0
m_wtb_rssi      0
m_wtw_rssi      0
heart_rate      0
distance        0
latitude        0
longitude       0
altitude        0
speed           0
m_usagestats    0
w_light         0
dtype: int64
📁 '../data/merged_df_cwj_filled.csv' 저장 완료


In [23]:

# 나머지 열들 선택
remaining_cols = [col for col in df.columns if col not in cols_to_check]

# 결측치 개수 확인
missing_counts = df[remaining_cols].isnull().sum()

# 결측치가 존재하는 열만 출력
missing_counts = missing_counts[missing_counts > 0]

print("📊 [cols_to_check 외] 결측치가 있는 열들:")
print(missing_counts)

# 결측치를 0으로 채워 저장
df_zero_filled = df.copy()
df_zero_filled[remaining_cols] = df_zero_filled[remaining_cols].fillna(0)
df_zero_filled.to_csv('../data/merged_df_cwj_tozero.csv', index=False)
print("✅ 결측치를 0으로 채운 파일 저장 완료: merged_df_cwj_tozero.csv")

# 결측치가 있는 행을 삭제하여 저장
df_dropna = df.copy()
df_dropna = df_dropna.dropna(subset=remaining_cols)
df_dropna.to_csv('../data/merged_df_cwj_delete.csv', index=False)
print("✅ 결측치 행 삭제 파일 저장 완료: merged_df_cwj_delete.csv")


📊 [cols_to_check 외] 결측치가 있는 열들:
Music                        3717
Vehicle                      3717
Motor vehicle (road)         3717
Outside, urban or manmade    3717
Outside, rural or natural    3717
                             ... 
Heavy metal                  3717
Double bass                  3717
Drum and bass                3717
String section               3717
Punk rock                    3717
Length: 517, dtype: int64
✅ 결측치를 0으로 채운 파일 저장 완료: merged_df_cwj_tozero.csv
✅ 결측치 행 삭제 파일 저장 완료: merged_df_cwj_delete.csv


In [26]:
# 결측치 개수 세기
missing_zero_filled = df_zero_filled.isnull().sum().sum()
missing_dropna = df_dropna.isnull().sum().sum()

print(f"df_zero_filled 결측치 총 개수: {missing_zero_filled}")
print(f"df_dropna 결측치 총 개수: {missing_dropna}")

print(df_zero_filled.shape, df_dropna.shape)

df_zero_filled 결측치 총 개수: 0
df_dropna 결측치 총 개수: 0
(99190, 541) (95473, 541)


# train, test 셋 준비

In [12]:
import pandas as pd


df_zero_filled = pd.read_csv('../data/merged_df_cwj_tozero.csv')
print("✅ df_zero_filled shape:", df_zero_filled.shape)
#df_zero_filled.head()

✅ df_zero_filled shape: (99190, 541)


In [13]:
from sklearn.model_selection import train_test_split

train_val_df = pd.read_csv('../data/ch2025_metrics_train.csv')

df_train, df_val = train_test_split(train_val_df, test_size=0.2, random_state=42, shuffle=True)

print("df_train.head():")
print(df_train.head(10))
print("\ndf_val.head():")
print(df_val.head(10))


df_val.shape, df_train.shape

df_train.head():
    subject_id  sleep_date lifelog_date  Q1  Q2  Q3  S1  S2  S3
24        id01  2024-07-24   2024-07-23   0   1   1   0   0   1
17        id01  2024-07-15   2024-07-14   0   1   0   0   1   1
66        id02  2024-08-19   2024-08-18   0   1   1   1   0   1
148       id04  2024-09-02   2024-09-01   0   0   1   1   1   0
249       id06  2024-07-03   2024-07-02   0   1   1   1   1   1
31        id01  2024-08-21   2024-08-20   0   0   1   0   0   1
84        id02  2024-09-24   2024-09-23   0   1   1   1   1   1
307       id07  2024-08-02   2024-08-01   0   0   0   0   0   0
406       id09  2024-08-24   2024-08-23   0   0   0   0   1   0
389       id09  2024-07-18   2024-07-17   1   0   0   1   1   1

df_val.head():
    subject_id  sleep_date lifelog_date  Q1  Q2  Q3  S1  S2  S3
407       id09  2024-08-25   2024-08-24   1   0   0   1   1   1
444       id10  2024-09-03   2024-09-02   0   0   0   0   0   0
117       id03  2024-09-07   2024-09-06   1   1   0   1   0   0
30     

((90, 9), (360, 9))

In [14]:
# Define metric columns
metrics = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']

# Split df_train and df_val for each metric
for metric in metrics:
    globals()[f'df_train_{metric.lower()}'] = df_train[['subject_id', 'sleep_date', 'lifelog_date', metric]].copy()
    globals()[f'df_val_{metric.lower()}'] = df_val[['subject_id', 'sleep_date', 'lifelog_date', metric]].copy()

# Display head of all 12 dataframes
for metric in metrics:
    print(f"df_train_{metric.lower()} head:")
    print(globals()[f'df_train_{metric.lower()}'].head())
    print(f"\ndf_val_{metric.lower()} head:")
    print(globals()[f'df_val_{metric.lower()}'].head())
    print("\n" + "-"*40 + "\n")

df_train_q1 head:
    subject_id  sleep_date lifelog_date  Q1
24        id01  2024-07-24   2024-07-23   0
17        id01  2024-07-15   2024-07-14   0
66        id02  2024-08-19   2024-08-18   0
148       id04  2024-09-02   2024-09-01   0
249       id06  2024-07-03   2024-07-02   0

df_val_q1 head:
    subject_id  sleep_date lifelog_date  Q1
407       id09  2024-08-25   2024-08-24   1
444       id10  2024-09-03   2024-09-02   0
117       id03  2024-09-07   2024-09-06   1
30        id01  2024-08-20   2024-08-19   0
415       id09  2024-09-03   2024-09-02   1

----------------------------------------

df_train_q2 head:
    subject_id  sleep_date lifelog_date  Q2
24        id01  2024-07-24   2024-07-23   1
17        id01  2024-07-15   2024-07-14   1
66        id02  2024-08-19   2024-08-18   1
148       id04  2024-09-02   2024-09-01   0
249       id06  2024-07-03   2024-07-02   1

df_val_q2 head:
    subject_id  sleep_date lifelog_date  Q2
407       id09  2024-08-25   2024-08-24   0
444    

# GRU

In [15]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

df_merged = df_zero_filled.copy()
# timestamp → datetime 변환
df_merged["timestamp"] = pd.to_datetime(df_merged["timestamp"])

# lifelog_date = 날짜(문자열) ; 2024-06-26 같은 형식
df_merged["lifelog_date"] = df_merged["timestamp"].dt.date.astype(str)

# -------- 2. feature / sensor column 정의 -----------
drop_cols = ["timestamp",                # 시간은 예측에 불필요
             "subject_id",               # 매치용
             "lifelog_date"] + [c for c in df_merged.columns if c.startswith("id")]  # one-hot id

sensor_cols = [c for c in df_merged.columns if c not in drop_cols]
# 하루 당 최대 144 타임스텝으로 패딩
MAX_SEQ_LEN = 144


# -------- 3. 시퀀스 묶는 함수 -----------
def build_sequences(df):
    """(subject_id, lifelog_date) → ndarray(seq_len, n_feat)"""
    seq_dict = {}
    for (sid, day), g in df.groupby(['subject_id', 'lifelog_date']):
        # 10-분 간격 보장 안될 수도 있으니 timestamp 기준 정렬
        g = g.sort_values('timestamp')
        x = g[sensor_cols].to_numpy(dtype=np.float32)

        # 길이 조정
        if len(x) > MAX_SEQ_LEN:          # 잘라내기
            x = x[:MAX_SEQ_LEN]
        if len(x) < MAX_SEQ_LEN:          # 0-패딩
            pad = np.zeros((MAX_SEQ_LEN - len(x), x.shape[1]), np.float32)
            x = np.vstack([x, pad])

        seq_dict[(sid, day)] = x          # shape = (144, n_feat)
    return seq_dict

sequence_dict = build_sequences(df_merged)
print("# total sequences built :", len(sequence_dict))

# ---------- 3. Train / Val Tensor 준비 ----------
def rows_to_tensors(df_label):
    X, y = [], []
    for _, row in df_label.iterrows():
        key = (row['subject_id'], row['lifelog_date'])
        if key not in sequence_dict:        # 누락된 날짜 skip
            continue
        X.append(sequence_dict[key])
        y.append([row['Q1']])               # binary -> shape (1,)
    return np.stack(X), np.array(y, dtype=np.float32)

X_train, y_train = rows_to_tensors(df_train_q1)
X_val,   y_val   = rows_to_tensors(df_val_q1)

print("Train tensor shape :", X_train.shape, y_train.shape)
print("Val   tensor shape :", X_val.shape,   y_val.shape)

# ---------- 4. Feature 정규화 ----------
scaler = StandardScaler().fit(X_train.reshape(-1, len(sensor_cols)))
def scale(x):
    orig_shape = x.shape
    x = scaler.transform(x.reshape(-1, len(sensor_cols)))
    return x.reshape(orig_shape)

X_train = scale(X_train)
X_val   = scale(X_val)

# ---------- 5. PyTorch Dataset ----------
class SleepDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):  return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = SleepDS(X_train, y_train)
val_ds   = SleepDS(X_val,   y_val)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False)

# ---------- 6. GRU 모델 ----------
class GRUModel(nn.Module):
    def __init__(self, n_features, hidden=64):
        super().__init__()
        self.gru = nn.GRU(input_size=n_features, hidden_size=hidden,
                          num_layers=1, batch_first=True)
        self.fc  = nn.Linear(hidden, 1)
    def forward(self, x):
        _, h = self.gru(x)          # h: (1,B,hidden)
        h = h.squeeze(0)            # (B,hidden)
        return torch.sigmoid(self.fc(h))    # (B,1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = GRUModel(len(sensor_cols)).to(device)
criterion = nn.BCELoss()
optim     = torch.optim.Adam(model.parameters(), lr=1e-3)

# ---------- 7. 학습 루프 ----------
EPOCHS = 13
for epoch in range(1, EPOCHS+1):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optim.zero_grad()
        loss.backward()
        optim.step()

    # --- val ---
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            prob = model(xb).cpu().numpy()
            preds.append(prob)
            trues.append(yb.numpy())
    preds = np.vstack(preds)
    trues = np.vstack(trues)
    y_hat = (preds >= 0.5).astype(int)
    f1 = f1_score(trues, y_hat, average='macro')
    print(f"Epoch {epoch:2d} | Val macro‑F1: {f1:.4f}")



# total sequences built : 700
Train tensor shape : (360, 144, 529) (360, 1)
Val   tensor shape : (90, 144, 529) (90, 1)
Epoch  1 | Val macro‑F1: 0.5876
Epoch  2 | Val macro‑F1: 0.5847
Epoch  3 | Val macro‑F1: 0.5553
Epoch  4 | Val macro‑F1: 0.5217
Epoch  5 | Val macro‑F1: 0.5207
Epoch  6 | Val macro‑F1: 0.5174
Epoch  7 | Val macro‑F1: 0.5375
Epoch  8 | Val macro‑F1: 0.5662
Epoch  9 | Val macro‑F1: 0.5574
Epoch 10 | Val macro‑F1: 0.5331
Epoch 11 | Val macro‑F1: 0.5869
Epoch 12 | Val macro‑F1: 0.5207
Epoch 13 | Val macro‑F1: 0.5694


In [9]:
metrics = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']

def rows_to_tensors(df_label, metric):
    X, y = [], []
    for _, row in df_label.iterrows():
        key = (row['subject_id'], row['lifelog_date'])
        if key not in sequence_dict:        # 누락된 날짜 skip
            continue
        X.append(sequence_dict[key])
        y.append([row[metric]])             # 원하는 metric 컬럼 사용
    return np.stack(X), np.array(y, dtype=np.float32)

'''
# metric별로 train/val tensor 생성 및 shape 출력
for metric in metrics:
    train_df = globals()[f'df_train_{metric.lower()}']
    val_df   = globals()[f'df_val_{metric.lower()}']
    X_train, y_train = rows_to_tensors(train_df, metric)
    X_val,   y_val   = rows_to_tensors(val_df, metric)
    print(f"[{metric}] Train tensor shape :", X_train.shape, y_train.shape)
    print(f"[{metric}] Val   tensor shape :", X_val.shape,   y_val.shape)
'''

'''
# ---------- 3. Train / Val Tensor 준비 ----------
def rows_to_tensors(df_label, metric):
    X, y = [], []
    for _, row in df_label.iterrows():
        key = (row['subject_id'], row['lifelog_date'])
        if key not in sequence_dict:        # 누락된 날짜 skip
            continue
        X.append(sequence_dict[key])
        y.append([row[metric]])               # binary -> shape (1,)
    return np.stack(X), np.array(y, dtype=np.float32)

X_train, y_train = rows_to_tensors(df_train_q2, metric)
X_val,   y_val   = rows_to_tensors(df_val_q2, metric)

print("Train tensor shape :", X_train.shape, y_train.shape)
print("Val   tensor shape :", X_val.shape,   y_val.shape)
'''

# ---------- 4. Feature 정규화 ----------
scaler = StandardScaler().fit(X_train.reshape(-1, len(sensor_cols)))
def scale(x):
    orig_shape = x.shape
    x = scaler.transform(x.reshape(-1, len(sensor_cols)))
    return x.reshape(orig_shape)

X_train = scale(X_train)
X_val   = scale(X_val)

# ---------- 5. PyTorch Dataset ----------
class SleepDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):  return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = SleepDS(X_train, y_train)
val_ds   = SleepDS(X_val,   y_val)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False)

# ---------- 6. GRU 모델 ----------
class GRUModel(nn.Module):
    def __init__(self, n_features, hidden=64):
        super().__init__()
        self.gru = nn.GRU(input_size=n_features, hidden_size=hidden,
                          num_layers=1, batch_first=True)
        self.fc  = nn.Linear(hidden, 1)
    def forward(self, x):
        _, h = self.gru(x)          # h: (1,B,hidden)
        h = h.squeeze(0)            # (B,hidden)
        return torch.sigmoid(self.fc(h))    # (B,1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = GRUModel(len(sensor_cols)).to(device)
criterion = nn.BCELoss()
optim     = torch.optim.Adam(model.parameters(), lr=1e-3)

# ---------- 7. 학습 루프 ----------
EPOCHS = 20
for metric in metrics:
    train_df = globals()[f'df_train_{metric.lower()}']
    val_df   = globals()[f'df_val_{metric.lower()}']
    X_train, y_train = rows_to_tensors(train_df, metric)
    X_val,   y_val   = rows_to_tensors(val_df, metric)
    print(f"[{metric}] Train tensor shape :", X_train.shape, y_train.shape)
    print(f"[{metric}] Val   tensor shape :", X_val.shape,   y_val.shape)
    for epoch in range(1, EPOCHS+1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            optim.zero_grad()
            loss.backward()
            optim.step()

        # --- val ---
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                prob = model(xb).cpu().numpy()
                preds.append(prob)
                trues.append(yb.numpy())
        preds = np.vstack(preds)
        trues = np.vstack(trues)
        y_hat = (preds >= 0.5).astype(int)
        f1 = f1_score(trues, y_hat, average='macro')
        print(f"Epoch {epoch:2d} | Val macro‑F1: {f1:.4f}")



[Q1] Train tensor shape : (360, 144, 529) (360, 1)
[Q1] Val   tensor shape : (90, 144, 529) (90, 1)
Epoch  1 | Val macro‑F1: 0.5264
Epoch  2 | Val macro‑F1: 0.5342
Epoch  3 | Val macro‑F1: 0.5179
Epoch  4 | Val macro‑F1: 0.5104
Epoch  5 | Val macro‑F1: 0.5031
Epoch  6 | Val macro‑F1: 0.5031
Epoch  7 | Val macro‑F1: 0.5187
Epoch  8 | Val macro‑F1: 0.5654
Epoch  9 | Val macro‑F1: 0.5654
Epoch 10 | Val macro‑F1: 0.5036
Epoch 11 | Val macro‑F1: 0.5676
Epoch 12 | Val macro‑F1: 0.5759
Epoch 13 | Val macro‑F1: 0.5264
Epoch 14 | Val macro‑F1: 0.5636
Epoch 15 | Val macro‑F1: 0.5342
Epoch 16 | Val macro‑F1: 0.5569
Epoch 17 | Val macro‑F1: 0.5537
Epoch 18 | Val macro‑F1: 0.5569
Epoch 19 | Val macro‑F1: 0.5486
Epoch 20 | Val macro‑F1: 0.5750
[Q2] Train tensor shape : (360, 144, 529) (360, 1)
[Q2] Val   tensor shape : (90, 144, 529) (90, 1)
Epoch  1 | Val macro‑F1: 0.5328
Epoch  2 | Val macro‑F1: 0.5403
Epoch  3 | Val macro‑F1: 0.5403
Epoch  4 | Val macro‑F1: 0.5759
Epoch  5 | Val macro‑F1: 0.5503


In [15]:
import numpy as np

Q1_list = [0.5264, 0.5342, 0.5179, 0.5104, 0.5031, 0.5031, 0.5187, 0.5654, 0.5654, 0.5036, 0.5676, 0.5759, 0.5264, 0.5636, 0.5342, 0.5569, 0.5537, 0.5569, 0.5486, 0.5750]
Q2_list = [0.5328, 0.5403, 0.5403, 0.5759, 0.5503, 0.5588, 0.5593, 0.5767, 0.6063, 0.5537, 0.5909, 0.5673, 0.6112, 0.6021, 0.5759, 0.5817, 0.5944, 0.5930, 0.5507, 0.5593]
Q3_list = [0.5878, 0.5680, 0.5750, 0.5588, 0.5588, 0.5750, 0.5853, 0.5661, 0.5786, 0.5750, 0.5503, 0.6121, 0.6000, 0.6277, 0.5971, 0.6156, 0.6021, 0.5680, 0.5840, 0.5840]
S1_list = [0.5930, 0.6027, 0.6021, 0.5726, 0.5750, 0.5817, 0.5750, 0.5971, 0.5750, 0.6063, 0.5750, 0.6000, 0.6000, 0.6184, 0.5909, 0.6021, 0.6092, 0.6404, 0.5840, 0.6250]
S2_list = [0.6000, 0.6000, 0.5840, 0.6000, 0.6000, 0.5840, 0.5840, 0.5840, 0.5840, 0.5840, 0.5840, 0.5840, 0.5840, 0.5840, 0.6000, 0.5840, 0.5840, 0.5840, 0.5840, 0.6000]
S3_list = [0.6000, 0.6000, 0.5840, 0.6000, 0.6156, 0.6063, 0.6063, 0.5909, 0.5909, 0.5930, 0.5846, 0.5746, 0.5673, 0.5944, 0.6250, 0.5909, 0.5588, 0.5750, 0.5840, 0.5750]
# 각 인덱스별로 6개 리스트의 값을 평균내어 f1_list에 저장
f1_list = [np.mean([lst[i] for lst in all_lists]) for i in range(len(Q1_list))]
all_lists = [Q1_list, Q2_list, Q3_list, S1_list, S2_list, S3_list]
max_idx = np.argmax(f1_list)
print("가장 높은 값의 인덱스:", max_idx)
print("가장 높은 값:", f1_list[max_idx])
print(f1_list)

가장 높은 값의 인덱스: 13
가장 높은 값: 0.5983666666666667
[np.float64(0.5733333333333334), np.float64(0.5742), np.float64(0.5672166666666667), np.float64(0.5696166666666667), np.float64(0.5671333333333334), np.float64(0.56815), np.float64(0.5714333333333333), np.float64(0.5800333333333333), np.float64(0.5833666666666667), np.float64(0.5692666666666667), np.float64(0.5754), np.float64(0.5856500000000001), np.float64(0.5814833333333334), np.float64(0.5983666666666667), np.float64(0.5871833333333333), np.float64(0.5885333333333334), np.float64(0.5837), np.float64(0.5862166666666666), np.float64(0.57255), np.float64(0.5863833333333334)]


# 실제 적용

In [2]:
import pandas as pd


df_zero_filled = pd.read_csv('../data/merged_df_cwj_tozero.csv')
print("✅ df_zero_filled shape:", df_zero_filled.shape)
#df_zero_filled.head()

✅ df_zero_filled shape: (99190, 541)


In [3]:
from sklearn.model_selection import train_test_split

trainset = pd.read_csv('../data/ch2025_metrics_train.csv')

testset = pd.read_csv('../data/ch2025_submission_sample.csv')


In [4]:
testset.head(10)

,subject_id,sleep_date,lifelog_date,Q1,Q2,Q3,S1,S2,S3
0,id01,2024-07-31,2024-07-30,0,0,0,0,0,0
1,id01,2024-08-01,2024-07-31,0,0,0,0,0,0
2,id01,2024-08-02,2024-08-01,0,0,0,0,0,0
3,id01,2024-08-03,2024-08-02,0,0,0,0,0,0
4,id01,2024-08-04,2024-08-03,0,0,0,0,0,0
5,id01,2024-08-06,2024-08-05,0,0,0,0,0,0
6,id01,2024-08-07,2024-08-06,0,0,0,0,0,0
7,id01,2024-08-09,2024-08-08,0,0,0,0,0,0
8,id01,2024-08-10,2024-08-09,0,0,0,0,0,0
9,id01,2024-08-12,2024-08-11,0,0,0,0,0,0


In [5]:
# Define metric columns
metrics = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']

# Split df_train and df_val for each metric
for metric in metrics:
    globals()[f'trainset_{metric.lower()}'] = trainset[['subject_id', 'sleep_date', 'lifelog_date', metric]].copy()

# Display head of all 12 dataframes
for metric in metrics:
    print(f"trainset_{metric.lower()} head:")
    print(globals()[f'trainset_{metric.lower()}'].head())


trainset_q1 head:
  subject_id  sleep_date lifelog_date  Q1
0       id01  2024-06-27   2024-06-26   0
1       id01  2024-06-28   2024-06-27   0
2       id01  2024-06-29   2024-06-28   1
3       id01  2024-06-30   2024-06-29   1
4       id01  2024-07-01   2024-06-30   0
trainset_q2 head:
  subject_id  sleep_date lifelog_date  Q2
0       id01  2024-06-27   2024-06-26   0
1       id01  2024-06-28   2024-06-27   0
2       id01  2024-06-29   2024-06-28   0
3       id01  2024-06-30   2024-06-29   0
4       id01  2024-07-01   2024-06-30   1
trainset_q3 head:
  subject_id  sleep_date lifelog_date  Q3
0       id01  2024-06-27   2024-06-26   0
1       id01  2024-06-28   2024-06-27   0
2       id01  2024-06-29   2024-06-28   0
3       id01  2024-06-30   2024-06-29   1
4       id01  2024-07-01   2024-06-30   1
trainset_s1 head:
  subject_id  sleep_date lifelog_date  S1
0       id01  2024-06-27   2024-06-26   0
1       id01  2024-06-28   2024-06-27   0
2       id01  2024-06-29   2024-06-28   1
3   

In [18]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


df_merged = df_zero_filled.copy()
# timestamp → datetime 변환
df_merged["timestamp"] = pd.to_datetime(df_merged["timestamp"])

# lifelog_date = 날짜(문자열) ; 2024-06-26 같은 형식
df_merged["lifelog_date"] = df_merged["timestamp"].dt.date.astype(str)

# -------- 2. feature / sensor column 정의 -----------
drop_cols = ["timestamp",                # 시간은 예측에 불필요
             "subject_id",               # 매치용
             "lifelog_date"] + [c for c in df_merged.columns if c.startswith("id")]  # one-hot id

sensor_cols = [c for c in df_merged.columns if c not in drop_cols]
# 하루 당 최대 144 타임스텝으로 패딩
MAX_SEQ_LEN = 144

# testset 불러오기
testset = pd.read_csv("../data/ch2025_submission_sample.csv")

# ---- 1. testset에 대해 시계열 시퀀스 구성 ----
def build_test_sequences(testset, sequence_dict):
    X_test = []
    key_list = []
    for idx, row in testset.iterrows():
        key = (row['subject_id'], row['lifelog_date'])
        key_list.append(key)
        if key in sequence_dict:
            X_test.append(sequence_dict[key])
        else:
            # 패딩 시퀀스 (데이터가 없는 경우)
            x_pad = np.zeros((MAX_SEQ_LEN, len(sensor_cols)), dtype=np.float32)
            X_test.append(x_pad)
    return np.stack(X_test), key_list



# ---- 2. metric별로 학습/예측/채우기 ----
metrics = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']

for metric in metrics:
    # 1) train/val set 만들기
    train_df = trainset[['subject_id', 'lifelog_date', metric]].copy()
    # (validation split 필요 없으면 전체를 train으로 사용)
    X_train, y_train = [], []
    for _, row in train_df.iterrows():
        key = (row['subject_id'], row['lifelog_date'])
        if key not in sequence_dict:
            continue
        X_train.append(sequence_dict[key])
        y_train.append([row[metric]])
    X_train, y_train = np.stack(X_train), np.array(y_train, dtype=np.float32)

    # 2) 정규화 (train 기준)
    scaler = StandardScaler().fit(X_train.reshape(-1, len(sensor_cols)))
    X_train = scaler.transform(X_train.reshape(-1, len(sensor_cols))).reshape(X_train.shape)
    
    # 3) test 시퀀스 생성 & 정규화
    X_test, key_list = build_test_sequences(testset, sequence_dict)
    X_test = scaler.transform(X_test.reshape(-1, len(sensor_cols))).reshape(X_test.shape)

    # 4) Dataset/dataloader 생성
    test_ds = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32))
    test_loader = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=False)

    # 5) 모델 초기화 및 학습 (train만)
    model = GRUModel(len(sensor_cols)).to(device)
    criterion = torch.nn.BCELoss()
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)

    EPOCHS = 13  # (더 늘려도 됨)
    train_tensor = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
    train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=64, shuffle=True)

    for epoch in range(EPOCHS):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            optim.zero_grad()
            loss.backward()
            optim.step()

    # 6) testset 예측
    model.eval()
    preds = []
    with torch.no_grad():
        for (xb,) in test_loader:
            xb = xb.to(device)
            prob = model(xb).cpu().numpy()
            preds.append(prob)
    preds = np.vstack(preds)
    if metric == "S1":  # S1이 3-class라면 softmax/argmax 등 따로 처리
        testset[metric] = preds.argmax(axis=1)  # (필요시 수정)
    else:
        testset[metric] = (preds >= 0.5).astype(int)

# ---- 3. 결과 저장 ----
testset[['subject_id', 'sleep_date', 'lifelog_date'] + metrics].to_csv("submission.csv", index=False)
print("submission.csv 파일 저장 완료")


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# 실제 submission을 위한 최종 코드

df_merged = df_zero_filled.copy()
# timestamp → datetime 변환
df_merged["timestamp"] = pd.to_datetime(df_merged["timestamp"])

# lifelog_date = 날짜(문자열) ; 2024-06-26 같은 형식
df_merged["lifelog_date"] = df_merged["timestamp"].dt.date.astype(str)

# -------- 2. feature / sensor column 정의 -----------
drop_cols = ["timestamp",                # 시간은 예측에 불필요
             "subject_id",               # 매치용
             "lifelog_date"] + [c for c in df_merged.columns if c.startswith("id")]  # one-hot id

sensor_cols = [c for c in df_merged.columns if c not in drop_cols]
# 하루 당 최대 144 타임스텝으로 패딩
MAX_SEQ_LEN = 144


# -------- 3. 시퀀스 묶는 함수 -----------
def build_sequences(df):
    """(subject_id, lifelog_date) → ndarray(seq_len, n_feat)"""
    seq_dict = {}
    for (sid, day), g in df.groupby(['subject_id', 'lifelog_date']):
        # 10-분 간격 보장 안될 수도 있으니 timestamp 기준 정렬
        g = g.sort_values('timestamp')
        x = g[sensor_cols].to_numpy(dtype=np.float32)

        # 길이 조정
        if len(x) > MAX_SEQ_LEN:          # 잘라내기
            x = x[:MAX_SEQ_LEN]
        if len(x) < MAX_SEQ_LEN:          # 0-패딩
            pad = np.zeros((MAX_SEQ_LEN - len(x), x.shape[1]), np.float32)
            x = np.vstack([x, pad])

        seq_dict[(sid, day)] = x          # shape = (144, n_feat)
    return seq_dict

sequence_dict = build_sequences(df_merged)
print("# total sequences built :", len(sequence_dict))

# ---------- 3. Train / Val Tensor 준비 ----------
def rows_to_tensors(df_label):
    X, y = [], []
    for _, row in df_label.iterrows():
        key = (row['subject_id'], row['lifelog_date'])
        if key not in sequence_dict:        # 누락된 날짜 skip
            continue
        X.append(sequence_dict[key])
        y.append([row['Q1']])               # binary -> shape (1,)
    return np.stack(X), np.array(y, dtype=np.float32)

X_train, y_train = rows_to_tensors(df_train_q1)
X_val,   y_val   = rows_to_tensors(df_val_q1)

print("Train tensor shape :", X_train.shape, y_train.shape)
print("Val   tensor shape :", X_val.shape,   y_val.shape)

# ---------- 4. Feature 정규화 ----------
scaler = StandardScaler().fit(X_train.reshape(-1, len(sensor_cols)))
def scale(x):
    orig_shape = x.shape
    x = scaler.transform(x.reshape(-1, len(sensor_cols)))
    return x.reshape(orig_shape)

X_train = scale(X_train)
X_val   = scale(X_val)

# ---------- 5. PyTorch Dataset ----------
class SleepDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):  return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = SleepDS(X_train, y_train)
val_ds   = SleepDS(X_val,   y_val)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False)

# ---------- 6. GRU 모델 ----------
class GRUModel(nn.Module):
    def __init__(self, n_features, hidden=64):
        super().__init__()
        self.gru = nn.GRU(input_size=n_features, hidden_size=hidden,
                          num_layers=1, batch_first=True)
        self.fc  = nn.Linear(hidden, 1)
    def forward(self, x):
        _, h = self.gru(x)          # h: (1,B,hidden)
        h = h.squeeze(0)            # (B,hidden)
        return torch.sigmoid(self.fc(h))    # (B,1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = GRUModel(len(sensor_cols)).to(device)
criterion = nn.BCELoss()
optim     = torch.optim.Adam(model.parameters(), lr=1e-3)

# ---------- 7. 학습 루프 ----------
EPOCHS = 13
for epoch in range(1, EPOCHS+1):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optim.zero_grad()
        loss.backward()
        optim.step()

    # --- val ---
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            prob = model(xb).cpu().numpy()
            preds.append(prob)
            trues.append(yb.numpy())
    preds = np.vstack(preds)
    trues = np.vstack(trues)
    y_hat = (preds >= 0.5).astype(int)
    f1 = f1_score(trues, y_hat, average='macro')
    print(f"Epoch {epoch:2d} | Val macro‑F1: {f1:.4f}")

